# This Notebook handles the merging of the input images in each subdirectory

All subdirectory images will be converted to uniform sized (1900x4300) greyscale image, and concatenated together in a predefined way 

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool

In [2]:
def parse_and_resize(image_path, sy = 4300, sx = 1900):
    #image = cv.cvtColor(plt.imread(image_path), cv.COLOR_RGB2GRAY)
    image = plt.imread(image_path)
    resized = cv.resize(image, (sx,sy), interpolation = cv.INTER_CUBIC)
    return resized

def merge(top_left, top_right, bottom_left, bottom_right):
    a = np.hstack((top_left, top_right))
    b = np.hstack((bottom_left, bottom_right))
    c = np.vstack((a,b))
    return c

#Parsing the images in each subdirectory, to combine them later one single image 
def load_files(root):
    content = {}
    for directory, subdir, files in os.walk(root):
        #Check each file..
        for file in files:
            #If it's a jpg...
            if '.tiff' in file:
                # Save the parent directory as a key in content dict
                dir_as_key = directory.replace(root,'')
                # If this is the first one, create a list to store the file paths
                if dir_as_key not in content.keys():
                    content[dir_as_key] = []
                # Add path to image to the subdirectory key
                #print('\tlooking at file', os.path.join(directory,file))
                content[dir_as_key].append(os.path.join(directory,file))
    return content
#Complete process to merge images in all subdirectory
def merge_all_subdir_images(root):
    collection = load_files(root)
    merged_images = {}
    for key in collection.keys():
        images = collection[key]
        merged_images[key] = merge(parse_and_resize(images[1]),
                                   parse_and_resize(images[3]),
                                   parse_and_resize(images[0]),
                                   parse_and_resize(images[2]))
        plt.imsave('train/{0}.jpg'.format(key), merged_images[key], cmap = 'gray')
        print('Finished parsing & merging for {0}!'.format(key))
        del merged_images[key]
 

In [5]:
%%time
root = 'cancers/'
merge_all_subdir_images(root)

error: OpenCV(3.4.3) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:255: error: (-2:Unspecified error) in function '__cdecl cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [7]:
plt.imread(root + '/subdir1/A_0002_1.LEFT_CC.jpg')

array([[189, 200, 202, ...,   1,   1,   1],
       [ 34,  40,  40, ...,   1,   1,   1],
       [  7,   7,   7, ...,   1,   1,   1],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)